In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
from costs import *
from proj1_helpers import *
from franksun_function import *
from helpers import *

In [4]:
def build_poly(x, degree):
    poly_row = lambda x: [x**j for j in range(degree + 1)]
    return np.array([poly_row(e) for e in x])

In [26]:
def standardize(x, y):
    cols = [4, 5, 6, 12, 26, 27, 28]
    
    # Get the column where the above columns are not -999
    index = []
    for i in range(len(x)):
        for col in cols:
            if x[i][col] != -999:
                index.append(i)
                break
    
    # Get rid of those columns
    std_tX = np.delete(x, index, axis=0)
    std_y = np.delete(y, index)
    return std_tX, std_y
    
    
    
    

In [28]:
std_tX, std_y = standardize(tX, y)
print("finished standardization")


lambdas = np.logspace(-4, 2, 20)

for lambda_ in lambdas:
#     print(std_tX.shape)
#     print(std_y.shape)
    weights = ridge_regression(std_y, std_tX, lambda_)
    print("Lamda: ", lambda_, " Performance: ", performance(weights, y, tX))



finished standardization
Lamda:  0.0001  Performance:  0.65056
Lamda:  0.000206913808111  Performance:  0.69318
Lamda:  0.000428133239872  Performance:  0.673088
Lamda:  0.00088586679041  Performance:  0.666032
Lamda:  0.00183298071083  Performance:  0.6690240000000001
Lamda:  0.00379269019073  Performance:  0.677952
Lamda:  0.00784759970351  Performance:  0.6780999999999999
Lamda:  0.0162377673919  Performance:  0.67874
Lamda:  0.0335981828628  Performance:  0.6785479999999999
Lamda:  0.0695192796178  Performance:  0.678592
Lamda:  0.143844988829  Performance:  0.6785840000000001
Lamda:  0.297635144163  Performance:  0.6785760000000001
Lamda:  0.615848211066  Performance:  0.678572
Lamda:  1.2742749857  Performance:  0.678572
Lamda:  2.63665089873  Performance:  0.6785760000000001
Lamda:  5.45559478117  Performance:  0.6785760000000001
Lamda:  11.2883789168  Performance:  0.67858
Lamda:  23.3572146909  Performance:  0.6785760000000001
Lamda:  48.3293023857  Performance:  0.67858400000

## Generate predictions and save ouput in csv format for submission:

In [5]:
DATA_TEST_PATH = '../../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [9]:
OUTPUT_PATH = '../../data/output.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)